In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.utils import resample
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import ensemble

In [2]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [3]:
def evaluate_model(X_train_res, Y_train_res, X_test,Y_test):
    
    model = Sequential()
# LSTM LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(30, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
    
    # OPTIMIZER SETTINGS
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

    # MODEL COMPILE
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train_res, Y_train_res, epochs=10, batch_size=512, verbose=1,validation_data=(X_test, Y_test))
                                                      #callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    return model

In [7]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, Y_train_res = sm.fit_resample(X_train, Y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(Y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0)))

Before OverSampling, counts of label '1': 202311
Before OverSampling, counts of label '0': 721368 

After OverSampling, the shape of train_X: (1442736, 63)
After OverSampling, the shape of train_y: (1442736,) 

After OverSampling, counts of label '1': 721368
After OverSampling, counts of label '0': 721368


In [4]:
data4 = pd.read_csv('data4.csv')

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data4.corr(),annot=True, linewidth=0.5)

In [8]:
X_train_res= np.expand_dims(X_train_res, axis=2)

In [9]:
X_train_res.shape

(1442736, 63, 1)

In [10]:
X_test = np.expand_dims(X_test, axis=2)

In [11]:
X_test.shape

(395863, 63, 1)

In [ ]:
Y_train_res.shape

In [ ]:
Y_test.shape

In [5]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [6]:
train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [18]:
n_splits = 20
members = list()

for _ in range(n_splits):
    model = evaluate_model(X_train_res, Y_train_res, X_test, Y_test)
    members.append(model)

Epoch 1/10
22543/22543 [==============================] - 1934s 86ms/step - loss: 0.4915 - accuracy: 0.7329 - val_loss: 0.4118 - val_accuracy: 0.8064
Epoch 2/10
22543/22543 [==============================] - 1927s 85ms/step - loss: 0.3274 - accuracy: 0.8413 - val_loss: 0.2514 - val_accuracy: 0.8814
Epoch 3/10
22543/22543 [==============================] - 1910s 85ms/step - loss: 0.2316 - accuracy: 0.8942 - val_loss: 0.1829 - val_accuracy: 0.9156
Epoch 4/10
22543/22543 [==============================] - 1930s 86ms/step - loss: 0.3265 - accuracy: 0.8382 - val_loss: 0.3164 - val_accuracy: 0.8536
Epoch 5/10
22543/22543 [==============================] - 1901s 84ms/step - loss: 0.2360 - accuracy: 0.8946 - val_loss: 0.1661 - val_accuracy: 0.9258
Epoch 6/10
22543/22543 [==============================] - 1978s 88ms/step - loss: 0.1726 - accuracy: 0.9247 - val_loss: 0.1635 - val_accuracy: 0.9251
Epoch 7/10
22543/22543 [==============================] - 1914s 85ms/step - loss: 0.1492 - accuracy:

KeyboardInterrupt: 

In [13]:
n_splits = 20
members = list()

for _ in range(n_splits):
    model = evaluate_model(X_train_res, Y_train_res, X_test, Y_test)
    members.append(model)

Epoch 1/10
2818/2818 [==============================] - 749s 264ms/step - loss: 0.5368 - accuracy: 0.6996 - val_loss: 2.1455 - val_accuracy: 0.3141
Epoch 2/10
2818/2818 [==============================] - 741s 263ms/step - loss: 0.4412 - accuracy: 0.7714 - val_loss: 0.4450 - val_accuracy: 0.7851
Epoch 3/10
2818/2818 [==============================] - 733s 260ms/step - loss: 0.3397 - accuracy: 0.8358 - val_loss: 0.2659 - val_accuracy: 0.8819
Epoch 4/10
2818/2818 [==============================] - 747s 265ms/step - loss: 0.2403 - accuracy: 0.8903 - val_loss: 0.2574 - val_accuracy: 0.8861
Epoch 5/10
2818/2818 [==============================] - 769s 273ms/step - loss: 0.1871 - accuracy: 0.9171 - val_loss: 0.1937 - val_accuracy: 0.9123
Epoch 6/10
2818/2818 [==============================] - 767s 272ms/step - loss: 0.1587 - accuracy: 0.9300 - val_loss: 0.2055 - val_accuracy: 0.9096
Epoch 7/10
2818/2818 [==============================] - 701s 249ms/step - loss: 0.1429 - accuracy: 0.9369 - val_

KeyboardInterrupt: 

In [12]:
def evaluate_model(X_train_res, Y_train_res, X_test,Y_test):
    
    model = Sequential()
# LSTM LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(30, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
    
    # OPTIMIZER SETTINGS
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

    # MODEL COMPILE
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,validation_data=(X_test, Y_test))
                                                      #callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    return model

In [13]:
n_splits = 20
members = list()

for _ in range(n_splits):
    model = evaluate_model(X_train_res, Y_train_res, X_test, Y_test)
    members.append(model)

Epoch 1/5
2818/2818 [==============================] - 1366s 481ms/step - loss: 0.5237 - accuracy: 0.7080 - val_loss: 4.6334 - val_accuracy: 0.2415
Epoch 2/5
2818/2818 [==============================] - 1327s 471ms/step - loss: 0.4114 - accuracy: 0.7903 - val_loss: 0.3998 - val_accuracy: 0.8016
Epoch 3/5
2818/2818 [==============================] - 1326s 470ms/step - loss: 0.2483 - accuracy: 0.8851 - val_loss: 0.1756 - val_accuracy: 0.9231
Epoch 4/5
2818/2818 [==============================] - 1315s 467ms/step - loss: 0.1705 - accuracy: 0.9246 - val_loss: 0.1476 - val_accuracy: 0.9312
Epoch 5/5
2818/2818 [==============================] - 1360s 482ms/step - loss: 0.1383 - accuracy: 0.9386 - val_loss: 0.1502 - val_accuracy: 0.9286
Epoch 1/5
2818/2818 [==============================] - 1282s 453ms/step - loss: 0.5241 - accuracy: 0.7099 - val_loss: 0.7252 - val_accuracy: 0.6260
Epoch 2/5
1847/2818 [==================>...........] - ETA: 6:44 - loss: 0.4344 - accuracy: 0.7752

KeyboardInterrupt: 

In [19]:
def evaluate_model(X_train_res, Y_train_res, X_test,Y_test):
    
    model = Sequential()
# LSTM LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
    model.add(BatchNormalization())
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(30, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
    
    # OPTIMIZER SETTINGS
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

    # MODEL COMPILE
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,validation_data=(X_test, Y_test))
                                                      #callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    return model

In [20]:
n_splits = 10
members = list()

for _ in range(n_splits):
    model = evaluate_model(X_train_res, Y_train_res, X_test, Y_test)
    members.append(model)

Epoch 1/5
2818/2818 [==============================] - 1996s 706ms/step - loss: 0.5235 - accuracy: 0.7089 - val_loss: 5.0595 - val_accuracy: 0.2411
Epoch 2/5
2818/2818 [==============================] - 2023s 718ms/step - loss: 0.3262 - accuracy: 0.8408 - val_loss: 0.2970 - val_accuracy: 0.8700
Epoch 3/5
2818/2818 [==============================] - 2282s 810ms/step - loss: 0.1648 - accuracy: 0.9263 - val_loss: 0.1613 - val_accuracy: 0.9264
Epoch 4/5
2818/2818 [==============================] - 2688s 954ms/step - loss: 0.1275 - accuracy: 0.9429 - val_loss: 0.1387 - val_accuracy: 0.9356
Epoch 5/5
2818/2818 [==============================] - 2969s 1s/step - loss: 0.1138 - accuracy: 0.9482 - val_loss: 0.1464 - val_accuracy: 0.9311
Epoch 1/5
 934/2818 [========>.....................] - ETA: 33:41 - loss: 0.5929 - accuracy: 0.6573

KeyboardInterrupt: 

In [21]:
def evaluate_model(X_train_res, Y_train_res, X_test,Y_test):
    
    model = Sequential()
# LSTM LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
    model.add(BatchNormalization())
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(30, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
    
    # OPTIMIZER SETTINGS
    opt = tf.keras.optimizers.Adam(learning_rate=0.0017, decay=1e-6)

    # MODEL COMPILE
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,validation_data=(X_test, Y_test))
                                                      #callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    return model

In [22]:
n_splits = 10
members = list()

for _ in range(n_splits):
    model = evaluate_model(X_train_res, Y_train_res, X_test, Y_test)
    members.append(model)

Epoch 1/5
2818/2818 [==============================] - 2047s 723ms/step - loss: 0.5289 - accuracy: 0.7040 - val_loss: 0.8287 - val_accuracy: 0.4791
Epoch 2/5
2818/2818 [==============================] - 2048s 727ms/step - loss: 0.3920 - accuracy: 0.7991 - val_loss: 0.7095 - val_accuracy: 0.7279
Epoch 3/5
2818/2818 [==============================] - 2039s 724ms/step - loss: 0.1797 - accuracy: 0.9190 - val_loss: 0.1306 - val_accuracy: 0.9369
Epoch 4/5
2818/2818 [==============================] - 2007s 712ms/step - loss: 0.1278 - accuracy: 0.9425 - val_loss: 0.1239 - val_accuracy: 0.9375
Epoch 5/5
2818/2818 [==============================] - 2219s 788ms/step - loss: 0.1158 - accuracy: 0.9472 - val_loss: 0.1045 - val_accuracy: 0.9450
Epoch 1/5
 382/2818 [===>..........................] - ETA: 28:07 - loss: 0.6338 - accuracy: 0.6161

KeyboardInterrupt: 